# Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.svm import SVR
from skforecast.recursive import ForecasterRecursive
from sklearn.linear_model import LinearRegression

# Load Dataset

In [2]:
# Load dataset
df = pd.read_csv('input.csv')
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


# Preprocessing

In [3]:
# Drop unnecessary columns
df = df.drop(['UDI'], axis=1)
df = df.drop(['Product ID'], axis=1)
df = df.drop(['Type'], axis=1)
df = df.drop(['Target'], axis=1)
df = df.drop(['Failure Type'], axis=1)
df.columns

Index(['Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]'],
      dtype='object')

In [4]:
df = df.rename(columns={'Air temperature [K]': 'Air temperature',
                                'Process temperature [K]': 'Process temperature',
                                'Rotational speed [rpm]': 'Rotational speed',
                                'Torque [Nm]': 'Torque',
                                'Tool wear [min]': 'Tool wear',
                                })
df.columns

Index(['Air temperature', 'Process temperature', 'Rotational speed', 'Torque',
       'Tool wear'],
      dtype='object')

In [5]:
def detect_outliers_iqr(df, column):    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Air temperature Forecasting

In [6]:
df = df.set_index("Tool wear")
full_index = pd.Index(range(df.index.max() + 1), name="Tool wear")
df = df.reindex(full_index)

_, lower_bound, upper_bound = detect_outliers_iqr(df, "Air temperature")
df.loc[df["Air temperature"] < lower_bound, 'Air temperature'] = lower_bound
df.loc[df["Air temperature"] > upper_bound, 'Air temperature'] = upper_bound

df["Air temperature"] = df["Air temperature"].interpolate()
df = df.reset_index(drop=True)
x = pd.DataFrame(list(df.index), columns=["Index"])
y = df["Air temperature"]

svr = SVR(kernel='rbf')
svr.fit(x, y)
y_pred = svr.predict(x)

In [7]:
train = y_pred

forecaster = ForecasterRecursive(
                 regressor = LinearRegression(),
                 lags      = 1
             )
forecaster.fit(y=pd.Series(train, name="Air temperature"))

steps = 300
predictions = forecaster.predict(steps=steps)
predictions[predictions < 294] = 294
predictions[predictions > 306] = 306
predictions = predictions.rename('Air temperature')
output_df = pd.DataFrame(predictions)
output_df

,Air temperature
209,298.895365
210,298.896536
211,298.897694
212,298.898839
213,298.899970
...,...
504,298.994674
505,298.994714
506,298.994754
507,298.994793


# Process temperature Forecasting

In [8]:
_, lower_bound, upper_bound = detect_outliers_iqr(df, "Process temperature")
df.loc[df["Process temperature"] < lower_bound, 'Process temperature'] = lower_bound
df.loc[df["Process temperature"] > upper_bound, 'Process temperature'] = upper_bound

process_temperature_ori = df["Process temperature"] - df["Air temperature"] - 10
df["Process temperature"] = process_temperature_ori.interpolate()
x = pd.DataFrame(list(df.index), columns=["Index"])
y = df["Process temperature"]

svr = SVR(kernel='rbf')
svr.fit(x, y)
y_pred = svr.predict(x)

In [9]:
train = y_pred
forecaster = ForecasterRecursive(
                 regressor = LinearRegression(),
                 lags      = 1
             )
forecaster.fit(y=pd.Series(train, name="Process temperature"))

steps = 300
predictions = forecaster.predict(steps=steps)
predictions[predictions < -3] = -3
predictions[predictions > 3] = 3
predictions = predictions.values + output_df["Air temperature"].values + 10
predictions = pd.Series(predictions, name='Process temperature', index=output_df.index)
output_df = pd.concat([output_df, predictions], axis=1)
output_df

,Air temperature,Process temperature
209,298.895365,308.995546
210,298.896536,308.994325
211,298.897694,308.993074
212,298.898839,308.991793
213,298.899970,308.990482
...,...,...
504,298.994674,306.781032
505,298.994714,306.762586
506,298.994754,306.744010
507,298.994793,306.725305


# Rotational speed Forecasting

In [10]:
_, lower_bound, upper_bound = detect_outliers_iqr(df, "Rotational speed")
df.loc[df["Rotational speed"] < lower_bound, 'Rotational speed'] = lower_bound
df.loc[df["Rotational speed"] > upper_bound, 'Rotational speed'] = upper_bound

df["Rotational speed"] = df["Rotational speed"].interpolate()
y = df["Rotational speed"]
y_pred = np.mean(y) * np.ones_like(y)

In [11]:
train = y_pred
forecaster = ForecasterRecursive(
                 regressor = LinearRegression(),
                 lags      = 1
             )
forecaster.fit(y=pd.Series(train, name="Rotational speed"))

steps = 300
predictions = forecaster.predict(steps=steps)
predictions = predictions.rename('Rotational speed')
output_df = pd.concat([output_df, predictions], axis=1)
output_df

,Air temperature,Process temperature,Rotational speed
209,298.895365,308.995546,1530.462321
210,298.896536,308.994325,1530.462321
211,298.897694,308.993074,1530.462321
212,298.898839,308.991793,1530.462321
213,298.899970,308.990482,1530.462321
...,...,...,...
504,298.994674,306.781032,1530.462321
505,298.994714,306.762586,1530.462321
506,298.994754,306.744010,1530.462321
507,298.994793,306.725305,1530.462321


# Torque Forecasting

In [12]:
_, lower_bound, upper_bound = detect_outliers_iqr(df, "Torque")
df.loc[df["Torque"] < lower_bound, 'Torque'] = lower_bound
df.loc[df["Torque"] > upper_bound, 'Torque'] = upper_bound

df["Torque"] = df["Torque"].interpolate()
y = df["Torque"]
y_pred = 40 * np.ones_like(y)

In [13]:
train = y_pred
forecaster = ForecasterRecursive(
                 regressor = LinearRegression(),
                 lags      = 1
             )
forecaster.fit(y=pd.Series(train, name="Torque"))

steps = 300
predictions = forecaster.predict(steps=steps)
predictions[predictions < 0] = 0
predictions = predictions.rename('Torque')
output_df = pd.concat([output_df, predictions], axis=1)
output_df

,Air temperature,Process temperature,Rotational speed,Torque
209,298.895365,308.995546,1530.462321,40.0
210,298.896536,308.994325,1530.462321,40.0
211,298.897694,308.993074,1530.462321,40.0
212,298.898839,308.991793,1530.462321,40.0
213,298.899970,308.990482,1530.462321,40.0
...,...,...,...,...
504,298.994674,306.781032,1530.462321,40.0
505,298.994714,306.762586,1530.462321,40.0
506,298.994754,306.744010,1530.462321,40.0
507,298.994793,306.725305,1530.462321,40.0


# Save CSV

In [14]:
indices = list(output_df.index)
indices = pd.Series(indices, name='Tool wear', index=output_df.index)
output_df = pd.concat([output_df, indices], axis=1)
output_df = output_df.reset_index(drop=True)
output_df.to_csv('forecaster_output.csv', index=False)
output_df

,Air temperature,Process temperature,Rotational speed,Torque,Tool wear
0,298.895365,308.995546,1530.462321,40.0,209
1,298.896536,308.994325,1530.462321,40.0,210
2,298.897694,308.993074,1530.462321,40.0,211
3,298.898839,308.991793,1530.462321,40.0,212
4,298.899970,308.990482,1530.462321,40.0,213
...,...,...,...,...,...
295,298.994674,306.781032,1530.462321,40.0,504
296,298.994714,306.762586,1530.462321,40.0,505
297,298.994754,306.744010,1530.462321,40.0,506
298,298.994793,306.725305,1530.462321,40.0,507
